In [ ]:
from dotenv import load_dotenv
import os
from crewai import Crew, Task, Agent
from crewai.tools import tool
from crewai.process import Process
from langchain_openai import ChatOpenAI
import yfinance as yf
from datetime import datetime
import requests


load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


class Tools:

    @tool("Get ticker from company name")
    def get_ticker_from_name(company_name: str) -> str:
        """
        Get the stock ticker from the company name.
        This function will return the ticker for a given company name.
        """
        return get_ticker(company_name)

    @tool("One month stock price history")
    def stock_price(ticker: str) -> str:
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        # csv_data = stock.history(period="1mo").to_csv()
        # ticker를 포함하여 반환
        # return {"ticker": ticker, "data": csv_data}
        return stock.history(period="1mo").to_csv()


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            # 주가를 가져오는 툴
            tools=[
                Tools.get_ticker_from_name,
                Tools.stock_price,
            ],
        )


class Tasks:

    def technical_analysis(self, agent, company_name):
        ticker = get_ticker(company_name)

        if ticker is None:
            ticker = company_name

        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            # context="Stock analysis for {company}. Use historical price data and tools to provide entry points and price targets.",
            agent=agent,
            tools=[Tools.get_ticker_from_name],
            # output_file="technical_analysis_test.md",
            output_file=self.get_output_file(ticker),
        )

    def get_output_file(self, ticker):
        today = datetime.today().strftime("%m%d%Y")
        output_folder = f"{ticker}_{today}"

        os.makedirs(output_folder, exist_ok=True)
        return f"{output_folder}/technical_analysis.md"


def get_ticker(company_name: str) -> str:
    """
    회사 이름을 입력하면 Yahoo Finance에서 티커(symbol)를 찾아 반환.
    """
    url = f"https://query2.finance.yahoo.com/v1/finance/search?q={company_name}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        ticker = data["quotes"][0]["symbol"]  # 첫 번째 검색 결과의 티커 반환
        return ticker
    except:
        return None  # 검색 결과 없음


agents = Agents()
tasks = Tasks()

technical_analyst = agents.technical_analyst()

company_name = "tesla mortors"


technical_task = tasks.technical_analysis(
    technical_analyst,
    company_name,
)

crew = Crew(
    agents=[
        technical_analyst,
    ],
    tasks=[
        technical_task,
    ],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)


result = crew.kickoff(
    inputs=dict(
        company=company_name,
    ),
)